1. Download

In [ ]:
#pip install opencv-python matplotlib tensorflow mediapipe scikit-learn

2. Import

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt    # MIGHT REMOVE
import time                             # MIGHT REMOVE
import mediapipe as mp

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard # MIGHT REMOVE

mp_pose = mp.solutions.pose
mp_draw = mp.solutions.drawing_utils

3. Functions

In [2]:
# Mediapipe Detection Function
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Color Conversion BGR to RGB
    image.flags.writeable = False                   # Non-writeable
    results = model.process(image)                  # Make Prediction
    image.flags.writeable = True                    # Writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Color Conversion RGB back to BGR
    return image, results

In [3]:
# Mediapipe Draw Function
def draw_styled_landmarks(image, results):
    mp_draw.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,                     # Draw Face
                              mp_draw.DrawingSpec(color=(255,255,255), thickness=1, circle_radius=1),   # Face Dot Color
                              mp_draw.DrawingSpec(color=(240,19,255), thickness=1, circle_radius=1),    # Face Line Color
                              )

In [4]:
# Extract Keypoints Function
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*3)
    return pose
    #return np.concatenate([pose, face, leftHand, rightHand])

In [5]:
colors = [(245,118,16),(117,245,16),(16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
    return output_frame

TEST Pose Detection

In [ ]:
cap = cv2.VideoCapture(0)
# Set Mediapipe Model
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose_model:
    while cap.isOpened():
        
        # Read Feed
        ret, frame = cap.read()
        
        # Make Detections
        image, results = mediapipe_detection(frame, pose_model)
        print(results)
        
        # Draw Landmarks
        draw_styled_landmarks(image, results)

        # Show Feed
        cv2.imshow('OpenCV Feed', image)

        # Break Loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # If Broken
    cap.release()
    cv2.destroyAllWindows()

4. Training

In [34]:
# Path for stored data
DATA_PATH = os.path.join('MP_DATA')

# Actions to detect
actions = np.array(['nothing', 'deadlift', 'squat'])

# Create Directory
for action in actions: 
    os.makedirs(os.path.join(DATA_PATH, action))

In [53]:
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose_model:
    while cap.isOpened():
        
        ret, frame = cap.read()
        image, results = mediapipe_detection(frame, pose_model)
        draw_styled_landmarks(image, results)

        cv2.putText(image, 'Collectiong frames for {}'.format(action), (0,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)

        cv2.imshow('Data Capture', image)

        key = cv2.waitKey(10) & 0xFF
        if key == ord('1'):
            action = actions[0]
        elif key == ord('2'):
            action = actions[1]
        elif key == ord('3'):
            action = actions[2]
        elif key == ord(' '):
            # Make New Directory
            name = str(len(os.listdir(os.path.join(DATA_PATH, action))))
            os.makedirs(os.path.join(DATA_PATH, action, name))

            for i in range(30):
                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, pose_model)
                print(results)
                draw_styled_landmarks(image, results)
                cv2.imshow('Data Capture', image)
                npy_path = os.path.join(DATA_PATH, action, name, str(i))
                keypoints = extract_keypoints(results)
                np.save(npy_path, keypoints)
        elif key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [50]:
res = np.load(os.path.join(DATA_PATH, 'nothing', '2', '98.npy'))


In [51]:
res

array([ 0.53691816,  0.73623657, -1.6380198 ,  0.56405675,  0.64742661,
       -1.5367924 ,  0.582147  ,  0.64732909, -1.53719664,  0.60168898,
        0.64865977, -1.53725648,  0.4941453 ,  0.65005088, -1.54873669,
        0.46642661,  0.65133214, -1.54926991,  0.44070768,  0.65518272,
       -1.54970551,  0.63141775,  0.68664956, -0.90053523,  0.4088265 ,
        0.6959399 , -0.94984037,  0.57361913,  0.82690829, -1.39206922,
        0.4892717 ,  0.83052713, -1.40909457,  0.77836215,  1.03536046,
       -0.62004721,  0.26042417,  1.02614939, -0.58204401,  0.91526425,
        1.29555845, -1.15138257,  0.16525671,  1.38284492, -1.08473992,
        0.78394479,  1.29778886, -1.91218567,  0.2769759 ,  1.16376829,
       -2.01656628,  0.7675246 ,  1.31514323, -2.06282187,  0.31254476,
        1.12163866, -2.19596267,  0.74010766,  1.26589191, -2.05550718,
        0.29877579,  1.06442285, -2.15962648,  0.73624855,  1.26024199,
       -1.93239498,  0.29609144,  1.080881  , -2.03685832,  0.67

In [ ]:
cap = cv2.VideoCapture(0)

# Set Mediapipe Model
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose_model:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
        
                # Read Feed
                ret, frame = cap.read()
                
                # Make Detections
                image, results = mediapipe_detection(frame, pose_model)
                print(results)
                
                # Draw Landmarks
                draw_styled_landmarks(image, results)

                # Apply Collection Logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120,200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collectiong frames for {} Video Number {}'.format(action, sequence), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000) # Starting frame break time
                else:
                    cv2.putText(image, 'Collection frames for {} Video Number {}'.format(action, sequence), (15,12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1, cv2.LINE_AA)
                
                # Export Keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Show Feed
                cv2.imshow('OpenCV Feed', image)

                # Break Loop
                if cv2.waitKey(10) & 0xFF == ord('s'):
                    break
                elif cv2.waitKey(10) & 0xFF == ord('q'):
                    print('q')

    # If Broken
    cap.release()
    cv2.destroyAllWindows()

In [20]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

In [ ]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
x = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.05)

In [ ]:
log_dir = os.path.join('logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,99)))    # Not sure 99 is the right amount yet
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy']) # Look up optimizer

In [ ]:
model.fit(x_train, y_train, epochs=2000, callbacks=[tb_callback]) # Be vary of the epochs, potentially stop early

In [ ]:
model.save('action.keras')

5. Test Model

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
# Look in other project when ready to test model

6. Live Predictions

In [ ]:
sequence = []
sentence = []
predicitons = []
threshold = 0.6

cap = cv2.VideoCapture(0)
# Set Mediapipe Model
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.8) as pose_model:
    while cap.isOpened():
        
        # Read Feed
        ret, frame = cap.read()
        
        # Make Detections
        image, results = mediapipe_detection(frame, pose_model)
        print(results)
        
        # Draw Landmarks
        draw_styled_landmarks(image, results)

        # Make Prediction
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predicitons.append(np.argmax(res))

        # Show Predictions
            if np.unique(predicitons[-10:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions [np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5]

        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ''.join(sentence), (3, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Show Probabilities
        image = prob_viz(res, actions, image, colors)

        # Show Feed
        cv2.imshow('OpenCV Feed', image)

        # Break Loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # If Broken
    cap.release()
    cv2.destroyAllWindows()